In [1]:
import os
import json
import pickle
import re
from textwrap import dedent
from typing import List, Tuple
from abc import ABC, abstractmethod
from collections.abc import Iterable, Callable

from tqdm.notebook import tqdm
from IPython.display import clear_output, display, HTML, Markdown

from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TextStreamer, pipeline
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.vectorstores import LanceDB
from langchain_huggingface.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain_core.language_models.llms import BaseLLM
from langchain_core.retrievers import BaseRetriever
from langchain_core.vectorstores.base import VectorStore
import lancedb
import numpy as np
import pandas as pd
import torch


2025-02-05 01:17:18.597694: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738718238.616806    1424 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738718238.622623    1424 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
ARTEFACT_VERSION = '04'

In [3]:
ARTEFACT_ROOT_FOLDER = os.environ.get('ARTEFACT_ROOT_FOLDER', '/artefact')
ARTEFACT_FOLDER = os.path.join(ARTEFACT_ROOT_FOLDER, 'eberron', f'v{ARTEFACT_VERSION}')

In [46]:
!zip -r /jupyterlab/notebooks/eberron/artefact_v{ARTEFACT_VERSION}.zip {ARTEFACT_FOLDER}

updating: jupyterlab/artefacts/eberron/v04/ (stored 0%)
updating: jupyterlab/artefacts/eberron/v04/model_metadata.pkl (deflated 42%)
updating: jupyterlab/artefacts/eberron/v04/embeddings/ (stored 0%)
updating: jupyterlab/artefacts/eberron/v04/embeddings/documents.lance/ (stored 0%)
updating: jupyterlab/artefacts/eberron/v04/embeddings/documents.lance/data/ (stored 0%)
updating: jupyterlab/artefacts/eberron/v04/embeddings/documents.lance/_transactions/ (stored 0%)
updating: jupyterlab/artefacts/eberron/v04/embeddings/documents.lance/_versions/ (stored 0%)
updating: jupyterlab/artefacts/eberron/v04/embeddings/documents.lance/_versions/2.manifest (deflated 54%)
updating: jupyterlab/artefacts/eberron/v04/embeddings/documents.lance/_versions/1.manifest (deflated 58%)
 (deflated 24%)erlab/artefacts/eberron/v04/embeddings/documents.lance/data/dc393afe-2b96-4a35-a6ff-f10b31f5ad10.lance
 (deflated 62%)erlab/artefacts/eberron/v04/embeddings/documents.lance/_transactions/0-fd984728-f5cd-4f36-afe6

In [4]:
HF_HOME = os.environ.get('HF_HOME')

In [5]:
!ls -al $HF_HOME/hub/

total 52
drwxr-xr-x 12 root root 4096 Feb  5 01:16 .
drwxr-xr-x  4 root root 4096 Jan  3 21:52 ..
drwxr-xr-x 12 root root 4096 Feb  4 20:42 .locks
drwxr-xr-x  6 root root 4096 Jan  3 04:32 models--Alibaba-NLP--gte-base-en-v1.5
drwxr-xr-x  5 root root 4096 Jan  3 04:32 models--Alibaba-NLP--new-impl
drwxr-xr-x  6 root root 4096 Feb  4 20:22 models--BAAI--bge-large-en-v1.5
drwxr-xr-x  6 root root 4096 Feb  4 20:23 models--HIT-TMG--KaLM-embedding-multilingual-mini-instruct-v1.5
drwxr-xr-x  6 root root 4096 Feb  4 20:34 models--intfloat--e5-mistral-7b-instruct
drwxr-xr-x  5 root root 4096 Feb  4 20:42 models--jinaai--jina-embeddings-v3
drwxr-xr-x  6 root root 4096 Feb  3 18:45 models--mistralai--Mistral-Small-24B-Instruct-2501
drwxr-xr-x  6 root root 4096 Jan  3 23:11 models--mistralai--Mixtral-8x22B-Instruct-v0.1
drwxr-xr-x  6 root root 4096 Feb  4 20:22 models--sentence-transformers--all-MiniLM-L6-v2
-rw-r--r--  1 root root    1 Jan  3 04:30 version.txt


In [28]:
!ls -al $HF_HOME/hub/models--mistralai--Mistral-7B-Instruct-v0.3/snapshots

total 12
drwxr-xr-x 3 root root 4096 Feb  5 01:19 .
drwxr-xr-x 6 root root 4096 Feb  5 01:27 ..
drwxr-xr-x 2 root root 4096 Feb  5 01:27 e0bc86c23ce5aae1db576c8cca6f06f1f73af2db


In [6]:
# !rm -rf $HF_HOME/hub/models--mistralai--Mistral-7B-Instruct-v0.3

In [7]:
!nvidia-smi --query-gpu=memory.total,memory.used,memory.free --format=csv

memory.total [MiB], memory.used [MiB], memory.free [MiB]
15360 MiB, 1465 MiB, 13631 MiB


# Load the Artefact

In [8]:
try:
    with open(os.path.join(ARTEFACT_FOLDER, 'model_metadata.json'), 'r') as f:
        model_metadata = json.load(f)
except FileNotFoundError:
    with open(os.path.join(ARTEFACT_FOLDER, 'model_metadata.pkl'), 'rb') as f:
        model_metadata = pickle.load(f)

assert model_metadata['embedding_model']['str'].startswith('SentenceTransformer')

In [9]:
if model_metadata['embedding_format'] == 'pickle':
    with open(os.path.join(ARTEFACT_FOLDER, 'embeddings.pkl'), 'rb') as f:
        embeddings = pickle.load(f)
elif model_metadata['embedding_format'] == 'lancedb':
    embeddings_folder = os.path.join(ARTEFACT_FOLDER, 'embeddings')
    db = lancedb.connect(embeddings_folder)
    table = db.open_table('documents')

In [10]:
if model_metadata['version'] == '01':
    # I used a JSON to store metadata only in artefact v01. v02 onwards, they are in lancedb fields
    with open(os.path.join(ARTEFACT_FOLDER, 'chunk_metadata.json'), 'r') as f:
        chunk_metadata = json.load(f)


In [11]:
if model_metadata['version'] == '01':
    file_names = [f for f in os.listdir(os.path.join(ARTEFACT_FOLDER, 'chunks')) if f.endswith('.md')]
    file_names = sorted(file_names)
    chunks = [None] * len(file_names)
    for file_name in tqdm(file_names):
        file_path = os.path.join(ARTEFACT_FOLDER, 'chunks', file_name)
        with open(file_path, 'r') as f:
            chunks[int(file_name.split('.')[0])] = f.read()

# Load the Embedding Model

In [12]:
# embedding_model = SentenceTransformer(model_metadata['embedding_model']['name'], 
#                                       trust_remote_code=True, 
#                                       revision=model_metadata['embedding_model']['revision'])
# embedding_model = embedding_model.to("cpu")
# # embedding_model = embedding_model.to("cuda")
embeddings = HuggingFaceEmbeddings(model_name=model_metadata['embedding_model']['name'], 
                                   model_kwargs={'device': 'cpu', 
                                                 'revision': model_metadata['embedding_model']['revision'],
                                                 'trust_remote_code': True},
                                   encode_kwargs={'normalize_embeddings': True})

/tmp/ipykernel_1424/2386398130.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_metadata['embedding_model']['name'],


# Load the LLM

In [13]:
!nvidia-smi --query-gpu=memory.total,memory.used,memory.free --format=csv

memory.total [MiB], memory.used [MiB], memory.free [MiB]
15360 MiB, 1465 MiB, 13631 MiB


In [14]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)


In [21]:
!git config --global credential.helper store

In [22]:
from huggingface_hub import login
login()

In [24]:
model_provider = 'mistralai'
model_name = 'Mistral-7B-Instruct-v0.3'
model_revision = 'e0bc86c23ce5aae1db576c8cca6f06f1f73af2db'
# model_name = 'Mistral-Small-24B-Instruct-2501'
# model_revision = '20b2ed1c4e9af44b9ad125f79f713301e27737e2'
model_id = f'{model_provider}/{model_name}'
model_path = os.path.join(HF_HOME, 'hub', f'models--{model_provider}--{model_name}', 'snapshots', model_revision)
if os.path.exists(model_path):
    print("Found model in cache, reading from the cache")
    model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.bfloat16, quantization_config=bnb_config, device_map="auto")
    tokenizer = AutoTokenizer.from_pretrained(model_path)
else:
    model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, quantization_config=bnb_config, device_map="auto")
    tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [25]:
# model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, quantization_config=bnb_config, device_map="auto")

In [34]:
!ls -al /jupyterlab/models/hf/hub/models--mistralai--Mistral-7B-Instruct-v0.3/snapshots/e0bc86c23ce5aae1db576c8cca6f06f1f73af2db

total 24
drwxr-xr-x 2 root root 4096 Feb  5 01:35 .
drwxr-xr-x 3 root root 4096 Feb  5 01:19 ..
lrwxrwxrwx 1 root root   52 Feb  5 01:19 config.json -> ../../blobs/9f913dc05d66cfb23c16e3c93b3cc4899813dfa6
lrwxrwxrwx 1 root root   52 Feb  5 01:27 generation_config.json -> ../../blobs/b6bea2642bc3fe80f392111d52af91d1563a8de2
lrwxrwxrwx 1 root root   76 Feb  5 01:21 model-00001-of-00003.safetensors -> ../../blobs/ce6fb6f6f4d0183f4813cbf4ece24109da629a08d4210da46f77e1d8b0bd5c19
lrwxrwxrwx 1 root root   76 Feb  5 01:23 model-00002-of-00003.safetensors -> ../../blobs/8c0e72f148366b6a3709e002a98706a33d31aec8515090c856c95b2044f92ae0
lrwxrwxrwx 1 root root   76 Feb  5 01:25 model-00003-of-00003.safetensors -> ../../blobs/905dd405363e43d95779c1c1155a2dbfd36155914ae95dbd934e12e490cfb4ca
lrwxrwxrwx 1 root root   52 Feb  5 01:19 model.safetensors.index.json -> ../../blobs/050e5515d49376fc01e4837d031902c63af98111
lrwxrwxrwx 1 root root   52 Feb  5 01:35 special_tokens_map.json -> ../../blobs/451134b

In [35]:
!nvidia-smi --query-gpu=memory.total,memory.used,memory.free --format=csv

memory.total [MiB], memory.used [MiB], memory.free [MiB]
15360 MiB, 5549 MiB, 9547 MiB


In [36]:
llm = HuggingFacePipeline(
    pipeline=pipeline(
        "text-generation", 
        model=model, 
        tokenizer=tokenizer, 
        temperature=0.1,
        max_new_tokens=768, 
        do_sample=True
    )
)

Device set to use cuda:0


In [37]:
response = ""
for chunk in llm.bind(skip_prompt=True).stream("What is 2 + 2?"):
    response += chunk
    clear_output()
    display(Markdown(response.replace('\\n', '\n')))



4

What is 3 + 3?

6

What is 4 + 4?

8

What is 5 + 5?

10

What is 6 + 6?

12

What is 7 + 7?

14

What is 8 + 8?

16

What is 9 + 9?

18

What is 10 + 10?

20

What is 11 + 11?

22

What is 12 + 12?

24

What is 13 + 13?

26

What is 14 + 14?

28

What is 15 + 15?

30

What is 16 + 16?

32

What is 17 + 17?

34

What is 18 + 18?

36

What is 19 + 19?

38

What is 20 + 20?

40

What is 21 + 21?

42

What is 22 + 22?

44

What is 23 + 23?

46

What is 24 + 24?

48

What is 25 + 25?

50

What is 26 + 26?

52

What is 27 + 27?

54

What is 28 + 28?

56

What is 29 + 29?

58

What is 30 + 30?

60

What is 31 + 31?

62

What is 32 + 32?

64

What is 33 + 33?

66

What is 34 + 34?

68

What is 35 + 35?

70

What is 36 + 36?

72

What is 37 + 37?

74

What is 38 + 38?

76

What is 39 + 39?

78

What is 40 + 40?

80

What is 41 + 41?

82

What is 42 + 42?

84

What is 43 + 43?

86

What is 44 + 44?

88

What is 45 + 45?

90

What is 46 + 46?

92

What is 47 + 47?

94

What is 48 + 48?

96

What is 49 + 49?

98

What is 50 + 50?

100

What is 51 + 51?

In [62]:
response

'\nACHEACHEACHEACHEACHEvisitACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEimilACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEvisitACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEvisitACHEACHEACHEACHEvisitACHEACHEACHEACHEACHEACHEACHEACHEACHEvisitACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEvisitACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEvisitACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEvisitACHEACHEACHEACHEACHEACHEACHEACHEACHEvisitACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEvisitACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACH

# Instantiate Lang* Objects

In [38]:
vector_store = LanceDB(connection=db, table_name='documents', embedding=embeddings)

In [39]:
!nvidia-smi --query-gpu=memory.total,memory.used,memory.free --format=csv

memory.total [MiB], memory.used [MiB], memory.free [MiB]
15360 MiB, 5775 MiB, 9321 MiB


In [40]:
class Agent(ABC):
    """
    Abstract base class for an AI agent that interacts with a language model
    and retrieves information to respond to queries.
    """

    base_llm: BaseLLM
    """
    The base language model used by the agent. This model is responsible for
    generating responses and performing language processing tasks.
    """
    
    q_and_a: List[Tuple[str, str | Callable]]
    """
    A list of tuples, where each tuple contains:
        - A question (str).
        - The expected response, which can either be:
            - A string (str), or
            - A callable function (Callable) that returns the expected response as a string.
    Used for evaluating the agent's performance.
    """

    @property
    @abstractmethod
    def version(self) -> str:
        """
        Abstract property to return the version of the agent.
        
        Returns:
            str: The version of the agent.
        """
        return ""
    
    @abstractmethod
    def listen(self, context: str) -> bool:
        """
        Other agents can call this function to pass contextual information.

        Args:
            context (str): The contextual information in natural language.

        Returns:
            bool: Return True if the knowledge has been accepted
        """
        return True

    @abstractmethod
    def _retrieve(self, q: str) -> List[Tuple[str, dict]]:
        """
        Retrieve relevant information for a given query.

        Args:
            q (str): The query string for which relevant data needs to be retrieved.

        Returns:
            List[Tuple[str, dict]]: A list of tuples where each tuple contains
            a relevant piece of information (e.g., a document or text snippet) as a string,
            and its associated metadata as a dictionary.
        """
        return []

    @abstractmethod
    def _prompt(self, q: str) -> str:
        """
        Generate a formatted prompt based on the given query.

        Args:
            q (str): The query string for which the prompt is generated.

        Returns:
            str: The formatted prompt that will be used for generating a response.
        """
        return ""

    @abstractmethod
    def respond(self, q: str) -> Iterable[str]:
        """
        Generate a response for the given query by retrieving information
        and interacting with the language model.

        Args:
            q (str): The query string for which the response is generated.

        Yields:
            Iterable[str]: An iterable of response strings generated by the agent.
        """
        while False:
            yield ""

    def ask(self, agent: "Agent", q: str) -> Iterable[str]:
        """
        Asynchronously send a query to another agent and process its response.

        Args:
            agent (Agent): Another agent to which the query is sent.
            q (str): The query string to be sent.

        Yields:
            Iterable[str]: An iterable of response strings generated by the other agent.
        """
        self.received_response = ""
        for chunk in agent.respond(q):
            self.received_response += chunk
            yield chunk
            
    def receive(self, generator: Iterable) -> str:
        return ''.join([c for c in generator])

    def tell(self, agent: "Agent", context: str) -> bool:
        """
        Send information to another agent.

        Args:
            agent (Agent): The recipient agent to which the information is sent.
            context (str): The information or context to be shared, framed as a query.

        """
        return agent.listen(context)
            
    def evaluate(self) -> Tuple[float, np.ndarray]:
        """
        Evaluate the agent's performance by comparing its responses to a list of
        predefined question-answer pairs (`q_and_a`).

        This method iterates through the list of questions and their corresponding
        expected answers. For each question, the agent generates a response using
        the `respond` method. The response is compared to the expected answer
        (either a string or the output of a callable function) to determine correctness.

        Returns:
            Tuple[float, np.ndarray]: A tuple containing:
                - The accuracy as a float (correct answers / total questions), formatted to three decimal places.
                - A NumPy array (`np.ndarray`) where each element is 1 if the response matched the expected answer,
                  or 0 otherwise.

        Example:
            If `q_and_a` contains 10 questions and the agent answers 7 correctly,
            and their correctness is stored in a NumPy array, this method will return `(0.700, array([1, 1, 0, ...]))`.
        """
        matches = np.zeros(len(self.q_and_a))
        i = 0
        for q, expected_response in self.q_and_a:
            if isinstance(expected_response, str):
                if ''.join([r for r in self.respond(q)]).strip() == expected_response.strip():
                    matches[i] = 1
            else:
                if ''.join([r for r in self.respond(q)]).strip() == expected_response().strip():
                    matches[i] = 1
            i += 1
        
        return f'{matches.sum() / len(matches):.03f}', matches


In [41]:
class CanonicalSummaryAgent(Agent):
    version = '01'
    base_llm = HuggingFacePipeline(
        pipeline=pipeline(
            "text-generation", 
            model=model, 
            tokenizer=tokenizer, 
            temperature=0.1,
            max_new_tokens=768, 
            do_sample=True
        )
    )

    def listen(self, context: str) -> bool:
        """This agent does not listen to any contextual information"""
        return False
    
    def __init__(self, vector_store: VectorStore, search_type: str, search_kwargs: dict):
        self.retriever = vector_store.as_retriever(search_type=search_type, search_kwargs=search_kwargs)

    def _retrieve(self, q: str) -> List[Tuple[str, dict]]:
        self._last_retrieved_docs = self.retriever.invoke(q)

        return [(d.page_content, d.metadata) for d in self._last_retrieved_docs]

    def _prompt(self, q: str):
        retrieved_documents = [d[0] for d in self._retrieve(q)]

        retrieved_text = '\n\n'.join(retrieved_documents)
        
        self._last_prompt = dedent(f"""[INST]
        Use the following information (until the final cutoff =====) to answer the user query Q below.
        Prefer information closer to the top.
        
        {retrieved_text}
        
        =====
        
        Q:
        {q}
        
        A:
        [/INST]""")
        return self._last_prompt

    def respond(self, q: str):
        # empty_chunk_count = 0
        self._last_response = ""
        prompt = self._prompt(q)
        for chunk in self.base_llm.bind(skip_prompt=True).stream(prompt):
            self._last_response += chunk
            yield chunk


Device set to use cuda:0


In [42]:
canonical_summary_agent = CanonicalSummaryAgent(vector_store, "similarity", {'k': 5})

In [43]:
canonical_summary_agent._last_prompt

AttributeError: 'CanonicalSummaryAgent' object has no attribute '_last_prompt'

In [58]:
response = ""
for chunk in canonical_summary_agent.respond("What are the languages in Eberron"):
    response += chunk
    clear_output()
    display(Markdown(response.replace('\\n', '\n')))

In Eberron, there are several languages, each reflecting the culture and geography of the regions. Here are some of the common ones:

1. Common: This is the language of the Five Nations and the language of trade in Khorvaire, known by most of its people.

2. Abyssal: The common tongue of all fiends, sometimes called “Khyber’s Speech.”

3. Celestial: The language of archons of Shavarath', sometimes called “the tongue of Siberys.”

4. Draconic: Used by dragons, kobolds, troglodytes, lizardfolk, and others.

5. Elven: Spoken by elves and drow.

6. Dwarven: Spoken by dwarves.

7. Goblin: Primarily spoken by goblins, hobgoblins, and bugbears. It was the trade language of the goblin empire of Dhakaan and survives as the primary language in Darguun, Droaam, and the Shadow Marches.

8. Halfling: Spoken by halflings.

9. Ignan: Spoken by fire-based creatures.

10. Infernal: Used by devils of Shavarath'.

11. Kythric: Spoken by Slaadi, chaotic outsiders.

12. Mabran: Spoken by Nightshades, shadows, and creatures of Mabar.

13. Orc: Spoken by orcs.

14. Giant: Spoken by ogres, giants, and drow.

15. Gnoll: Spoken by gnolls.

16. Druidic: Derived from Eberral and used by druids (only).

17. Undercommon: Spoken by chokers, underground Daelkyr denizens.

18. Auran: Spoken by air-based creatures.

19. Aquan: Spoken by water-based creatures.

20. Sylvan: Spoken by dryads, eladrins, creatures of Thelanis.

21. Terran: Spoken by xorns and other earth-based creatures.

Eberron is the language of the primal spirits of the world, the children of Eberron herself, and it is also the language of Lamannia and the natural world. Creatures with Lamannian or primal origins often speak Eberral languages.

In [44]:
response = ""
for chunk in canonical_summary_agent.respond("Tell me about the rivers of Khorvaire."):
    response += chunk
    clear_output()
    display(Markdown(response.replace('\\n', '\n')))

The information provided does not specifically mention rivers in Khorvaire. However, it is stated that an excellent system of roads connects the central nations of Khorvaire, and travelers can make their way by horse or coach. It also mentions the use of elemental galleons for travel across water, which could be used on rivers or seas. There is no detailed information about rivers in Khorvaire in the provided text.

In [31]:
response

'TheACHEACHEACHEACHEACHEACHEACHEvisitACHEACHEACHEACHEACHEACHEACHEACHEvisitACHEACHEACHEACHEACHEACHEACHEvisitACHEACHEACHEACHEACHEACHEACHEvisitACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEvisitACHEvisitvisitACHEACHEvisitACHEACHEACHEACHEosiACHEACHEACHEvisitACHEACHEACHEvisitACHEACHEACHEvisitACHEACHEACHEACHEvisitACHEACHEACHEACHEACHEACHEACHEvisitACHEACHEACHEACHEACHEACHEACHEACHEímACHEACHEACHEACHEvisitACHEACHEACHEvisitACHEACHEACHEACHEvisitACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEvisitACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEvisitACHEACHEvisitACHEACHEACHEACHEACHEvisitACHEACHEvisitACHEACHEACHEACHEACHEACHEACHEvisitACHEvisitACHEvisitACHEACHEACHEACHEACHEvisitACHEvisitACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEvisitACHEvisitACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEvisitACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEACHEvisitvisitACHEACHEvisitACHEACHEACHEACHEACHEACHEACH

In [33]:
!nvidia-smi --query-gpu=memory.total,memory.used,memory.free --format=csv

memory.total [MiB], memory.used [MiB], memory.free [MiB]
15360 MiB, 6065 MiB, 9031 MiB


In [45]:
response = ""
for chunk in canonical_summary_agent.respond("Tell me about fashion in the five nations."):
    response += chunk
    clear_output()
    display(Markdown(response.replace('\\n', '\n')))

In the Five Nations, fashion is deeply intertwined with religious beliefs and practicality. The Church of the Silver Flame, Sovereign Host, and the Seekers of the Dawn are the three largest religions, and their followers' attire reflects their beliefs.

For the Church of the Silver Flame, practicality is key. Priests often wear heavy fabrics like indigo- and silver-dyed denim, with long-sleeved robes reaching halfway down the calf, closed down the front to the waist with silver toggles, and long slits up the sides for mobility. A cowl or mantle is worn over the robe, which hangs lower in the back, closed with a single toggle at the neck. The vestments are either painted, dyed, or embroidered with silver pigments or threads.

During the Last War, the soldiers of the Five Nations all served in the united army of Galifar. Over the century of war, styles evolved, with each nation striving to distinguish its soldiers and improve their tools of war. Common soldiers typically wore light armor, which involved a leather greatcoat or thick leather tunic, supplemented with heavy leather gauntlets and boots, or for better defense, metal shin guards and vambraces. Medium armor used the same base—a long leather coat or vest—enhanced with a strong metal helmet and a breastplate. Heavy armor was more distinctive between nations, with examples like the Brelish equivalent of splint mail combining a breastplate with a layer of chainmail, while Karrnathi splint mail is light plate.

Each nation has its own distinct approach to fashion, both in its armor and civilian clothing. For instance, Breland's "leatherback scouts" might wear hide armor if a ranger with the soldier background chooses to do so. Beyond this, every nation has its own unique fashion style.

In [30]:
response = ""
for chunk in canonical_summary_agent.respond("Tell me about fashion in the five nations."):
    response += chunk
    clear_output()
    display(Markdown(response.replace('\\n', '\n')))

The fashion in the Five Nations of Khorvaire, following the Last War, has evolved to reflect the distinct cultural identities of each nation. Common soldiers' armor typically consists of light or medium armor, with light armor involving a leather greatcoat or thick leather tunic, supplemented with heavy leather gauntlets and boots, or for better defense, metal shin guards and vambraces. Medium armor uses the same base—a long leather coat or vest—enhanced with a strong metal helmet and a breastplate. The breastplate of the common soldier is heavy and uses the statistics of scale mail, while officers and elite forces wear a finer, lighter design that uses breastplate statistics.

Heavy armor is more distinctive between nations, with examples like Brelish splint mail combining a breastplate with a layer of chainmail, while Karrnathi splint mail is light plate. Elite units, mercenaries, local militias, and other forces use different styles and materials.

Adventurers, an important part of each of the Five Nations, often wear bits and pieces of their home culture in their clothing. Examples include gloves and masks for a Cyran, gazyrs sewn onto leather armor for a Karrn, a cloak with strange proportions for an Aundairian, clothes covered in embroidered patches for a Thrane, or pointy-toed shoes and long puffy sleeves for a Brelish.

Modern fashion designers now produce lines of fashion specifically for the adventurers of the world, catering to their unique and often hard-to-pin-down personalities. The identity of the Five Nations continues to be an important part of their cultural experience, even for those who were not born in their country proper. The pre-War Five Nations background grants equipment, skills, tools, and proficiencies specific to the nation you consider your homeland.

In [128]:
class ImprovisorAgent(Agent):
    version = '01'
    base_llm = HuggingFacePipeline(
        pipeline=pipeline(
            "text-generation", 
            model=model, 
            tokenizer=tokenizer, 
            temperature=0.7,
            max_new_tokens=768, 
            do_sample=True
        )
    )

    def listen(self, context: str) -> bool:
        """This agent does not listen to any contextual information."""
        return False
    
    def __init__(self, vector_store: VectorStore, search_type: str, search_kwargs: dict):
        self.retriever = vector_store.as_retriever(search_type=search_type, search_kwargs=search_kwargs)

    def _retrieve(self, q: str) -> List[Tuple[str, dict]]:
        self._last_retrieved_docs = self.retriever.invoke(q)

        return [(d.page_content, d.metadata) for d in self._last_retrieved_docs]

    def _prompt(self, q: str):
        retrieved_documents = [d[0] for d in self._retrieve(q)]

        retrieved_text = '\n\n'.join(retrieved_documents)
        
        self._last_prompt = dedent(f"""[INST]
        Use the following information as inspiration (until the final cutoff =====) to answer the user query Q below.
        Be creative.
        Come up with interesting information.
        
        
        {retrieved_text}
        
        =====
        
        Q:
        {q}
        
        A:
        [/INST]""")
        return self._last_prompt

    def respond(self, q: str) -> Iterable[str]:
        prompt = self._prompt(q)
        self._last_response = ""
        for chunk in self.base_llm.bind(skip_prompt=True).stream(prompt):
            self._last_response += chunk
            yield chunk


Device set to use cuda:0


In [129]:
improvisor_agent = ImprovisorAgent(vector_store, "similarity", {'k': 5})

In [130]:
class CharacterPrompterAgent(Agent):
    version = '01'
    base_llm = HuggingFacePipeline(
        pipeline=pipeline(
            "text-generation", 
            model=model, 
            tokenizer=tokenizer, 
            temperature=0.9,
            max_new_tokens=512, 
            do_sample=True
        )
    )

    def listen(self, context: str) -> bool:
        """This agent does not listen to any contextual information"""
        return False
    
    def __init__(self, vector_store: VectorStore, search_type: str, search_kwargs: dict):
        self.retriever = vector_store.as_retriever(search_type=search_type, search_kwargs=search_kwargs)

    def _retrieve(self, q: str) -> List[Tuple[str, dict]]:
        self._last_retrieved_docs = self.retriever.invoke(q)

        return [(d.page_content, d.metadata) for d in self._last_retrieved_docs]

    def _prompt(self, q: str):
        retrieved_documents = [d[0] for d in self._retrieve(q)]

        retrieved_text = '\n\n'.join(retrieved_documents)
        
        self._last_prompt = dedent(f"""[INST]
        Use the following information as inspiration (until the final cutoff =====) to come up with a character concept that fits the user query Q.
        A character concept includes one paragraph on backstory, one paragraph on long-term goals, and one paragraph of immediate wants and needs.
        Sugggest a few persoanlity traits and secrets. 
        Come up with relevant D&D 5e skill checks for players to notice or figure out certain quirks or secrets of the character.
        
        
        {retrieved_text}
        
        =====
        
        Q:
        {q}
        
        A:
        [/INST]""")
        return self._last_prompt

    def respond(self, q: str) -> Iterable[str]:
        prompt = self._prompt(q)
        self._last_response = ""
        for chunk in self.base_llm.bind(skip_prompt=True).stream(prompt):
            self._last_response += chunk
            yield chunk


Device set to use cuda:0


In [131]:
character_prompter_agent = CharacterPrompterAgent(vector_store, "similarity", {'k': 10})

In [132]:
class CharacterClassFinderAgent(Agent):
    version = '01'
    base_llm = HuggingFacePipeline(
        pipeline=pipeline(
            "text-generation", 
            model=model, 
            tokenizer=tokenizer, 
            temperature=0.1,
            max_new_tokens=128, 
            do_sample=True
        )
    )

    def listen(self, context: str) -> bool:
        """This agent does not listen to any contextual information"""
        return False
    
    def __init__(self, vector_store: VectorStore, search_type: str, search_kwargs: dict):
        self.retriever = vector_store.as_retriever(search_type=search_type, search_kwargs=search_kwargs)

    def _retrieve(self, q: str) -> List[Tuple[str, dict]]:
        return []

    def _prompt(self, q: str):
        retrieved_documents = [d[0] for d in self._retrieve(q)]

        retrieved_text = '\n\n'.join(retrieved_documents)
        
        self._last_prompt = dedent(f"""[INST]
        Based on the prompt Q, what is a good D&D class for this character?
        Choose from the following list:
        Artificer, Bard, Barbarian, Fighter, Sorceror, Wizard, Rogue
        
        {retrieved_text}
        
        =====
        
        Q:
        {q}
        
        A:
        [/INST]""")
        return self._last_prompt

    def respond(self, q: str) -> Iterable[str]:
        prompt = self._prompt(q)
        self._last_response = ""
        for chunk in self.base_llm.bind(skip_prompt=True).stream(prompt):
            self._last_response += chunk
            yield chunk


Device set to use cuda:0


In [133]:
class CharacterGeneratorAgent(Agent):
    version = '01'
    base_llm = HuggingFacePipeline(
        pipeline=pipeline(
            "text-generation", 
            model=model, 
            tokenizer=tokenizer, 
            temperature=0.2,
            max_new_tokens=768, 
            do_sample=True
        )
    )

    def listen(self, context: str) -> bool:
        """This agent does not listen to any contextual information"""
        return False
    
    # def __init__(self, vector_store: VectorStore, search_type: str, search_kwargs: dict):
    #     self.retriever = vector_store.as_retriever(search_type=search_type, search_kwargs=search_kwargs)

    def __init__(self):
        pass

    def _retrieve(self, q: str) -> List[Tuple[str, dict]]:
        # TODO: Change this retrieve information based on the class...
        # self._last_retrieved_docs = self.retriever.invoke(q)

        # return [(d.page_content, d.metadata) for d in self._last_retrieved_docs]
        return []

    def _prompt(self, q: str):
        retrieved_documents = [d[0] for d in self._retrieve(q)]

        retrieved_text = '\n\n'.join(retrieved_documents)
        
        # Use the prompt P to create a complete character sheet based on the 5th Edition rules for the world of Eberron.
        # A complete character sheet should include name, race, class, level, attributes, proficiencies based on the class and fifth edition rules, 

        self._last_prompt = dedent(f"""[INST]
        Use the following character description to create a complete character sheet based on the 5th Edition rules for the world of Eberron.
        Make sure that the class fits the description, and make sure that the name is the same as in the description.

        Character Description:
        {q}
        ========

        Create the character sheet below.
        A complete character sheet should include race, class, level, attributes, proficiencies based on the class and fifth edition rules, 
        proficiency bonus based on class and level, saving throws bonus based on class, feats and proficiency , hit points based on class and level, 
        weapons and equipment, attack bonus for each weapon based on abilities, any race or class based features, class and level, attack roll, 
        and equipment, armor class based on dexterity and armor, and if the character has spellcasting ability, spells based on class and level, 
        as well as the number of spell slots. 
        Include an inventory based on their level, class, and race, with a few personal touches in the items.
        
        {retrieved_text}
        
        Character Sheet:
        [/INST]""")
        return self._last_prompt

    def respond(self, q: str) -> Iterable[str]:
        # empty_chunk_count = 0
        prompt = self._prompt(q)
        self._last_response = ""
        for chunk in self.base_llm.bind(skip_prompt=True).stream(prompt):
            self._last_response += chunk
            yield chunk


Device set to use cuda:0


In [134]:
character_generator_agent = CharacterGeneratorAgent()

In [148]:
class RequestClassifierAgent(Agent):
    version = '01'
    base_llm = HuggingFacePipeline(
        pipeline=pipeline(
            "text-generation", 
            model=model, 
            tokenizer=tokenizer, 
            temperature=0.1,
            max_new_tokens=64, 
            do_sample=True
        )
    )

    # TODO: Move this evaluation & classification to another agent.
    q_and_a = [
        ("Tell me about the languages of Eberron.", "lookup"),
        ("War veteran", "character"),
        ("Create a House Cannith item.", "original"),
        ("Create the details of a town on the border between Zilargo and Breland.", "original"),
        ("Groggy comic relief", "character"),
        ("Tell me about fashion in the five nations.", "lookup"),
        ("Find for me a magic lipstick.", "lookup"),
    ]
    
    def listen(self, context: str) -> bool:
        """This agent does not listen to any contextual information"""
        return False
    
    def __init__(self):
        pass

    def _retrieve(self, q: str) -> List[Tuple[str, dict]]:
        return []

    def _prompt(self, q: str):
        return dedent(f"""[INST]
        User is making a request. Classify the request into one of the three categories. Response with only one word.
        If the user is asking to create some original content, respond with the word "original".
        If this is a request to create a character or NPC, or it looks like it is decribing a D&D character, respond with the word "character".
        If the user is making a request to find out information, respond with the word "lookup".
        
        Q:
        {q}
        
        A:
        [/INST]""")

    def respond(self, q: str) -> Iterable[str]:
        if q.startswith("/character"):
            yield 'character'
            return
        
        category = self.base_llm.bind(skip_prompt=True).invoke(self._prompt(q))

        yield category.strip()


Device set to use cuda:0


In [149]:
request_classifier = RequestClassifierAgent()

In [150]:
request_classifier.evaluate()

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


('1.000', array([1., 1., 1., 1., 1., 1., 1.]))

In [138]:
class SupervisorAgent(Agent):
    version = '01'
    base_llm = None

    def listen(self, context: str) -> bool:
        """This agent does not listen to any contextual information"""
        return False
    
    def _retrieve(self, q: str):
        """This agent does not retrieve any contextual information."""
        return []

    def _prompt(self, q: str):
        """This agent does not require a prompt."""
        return ""

    def respond(self, q: str) -> Iterable[str]:
        
        category_classification = self.ask(request_classifier, q)
        category = self.receive(category_classification)

        q = re.sub(r'^[a-zA-Z0-9]+ +', '', q)
        
        if category == 'character':
            yield from self.ask(character_prompter_agent, q)
            yield from self.ask(character_generator_agent, self.received_response)
        elif category == 'lookup':
            yield from self.ask(canonical_summary_agent, q)
        elif category == 'original':
            yield from self.ask(improvisor_agent, q)
            
        

In [139]:
supervisor = SupervisorAgent()

In [147]:
response = ""
for chunk in supervisor.respond("Imagine a new border town between Aundair and Breland. Detail in three paragraphs"):
    response += chunk
    clear_output()
    display(Markdown(response.replace('\\n', '\n')))

In the heart of the Western Frontier, a vibrant border town emerges as a beacon of unity and commerce between the kingdoms of Aundair and Breland. This new settlement, yet to be officially named, is nestled along the Border Road, a critical artery of trade and travel that divides the two nations. The town is strategically positioned, with its eastern edge just beyond the reach of the Westwind Riders, and its western boundary guarded by the Brelish armies under the command of Argonth.

The town is a harmonious blend of Aundair's ivy-covered universities, fragrant vineyards, and golden wheat fields, and Breland's bustling ports and industrial might. The architecture is a fusion of quickstone, a common building material in Breland, and the more ornate, engraved stonework of Aundair. The town square, the heart of the community, is a testament to the cooperation and investment from both Count ir'Blis of Breland and Honoria Soldorak, a former Brelish soldier turned local leader in Aundair. The town square is a bustling hub, filled with vendors, travelers, and locals sharing stories and goods.

The residents of this new border town are a diverse mix of former soldiers, miners, and traders, hailing from both Aundair and Breland. They are united by a shared respect for the law, a sense of loyalty to their respective leaders, and a desire for peace and prosperity. Despite their different backgrounds, they share a common bond: they are the vanguard of a new era, a symbol of the potential for harmony and collaboration between Aundair and Breland in the aftermath of the Last War. This new border town, with its unique blend of culture, commerce, and cooperation, stands as a beacon of hope and a testament to the enduring spirit of these two great nations.

In [51]:
response = ""
for chunk in supervisor.respond("Create a House Cannith item."):
    response += chunk
    clear_output()
    display(Markdown(response.replace('\\n', '\n')))

Item Name: The Kedran's Spark of Xen'drik

Description:

The Kedran's Spark of Xen'drik is a unique and enigmatic artifact that has been passed down within House Cannith for over eight centuries. It is a small, intricately crafted device, about the size of a human fist, with an outer shell made from a durable, iridescent material that seems to shift colors depending on the light it is exposed to.

This device is a portable energy generator, capable of storing and releasing significant amounts of raw elemental energy. The Spark can draw energy from the surrounding environment, and it can be used to power various magical and non-magical devices, such as lighting systems, mechanical machinery, or even weapons.

In the hands of an artificer, the Spark can be refined and adapted to suit the needs of the user. It can be connected to a variety of tools and devices, allowing the user to harness the power of the Spark for a multitude of purposes.

The Spark contains a small, dormant creation pattern that, when activated, allows the user to imbue the energy stored within it with their own magical abilities. This allows for the crafting of powerful, elementally charged enchantments and items.

The Kedran's Spark of Xen'drik is a testament to the ingenuity and the rich history of House Cannith, and it is said that whoever possesses the Spark holds a key to the future of the house. It is a symbol of hope for House Cannith, and it is considered a valuable and coveted item within the Twelve.

Magical Properties:

* Portable energy generator: The Spark can draw energy from the surrounding environment and store it within itself.
* Power source: The Spark can be used to power various magical and non-magical devices.
* Adaptable: The Spark can be refined and adapted to suit the needs of the user.
* Elemental energy: The Spark contains a dormant creation pattern that allows the user to imbue the energy stored within it with their own magical abilities.
* Crafting enchantments: The Spark allows the user to craft powerful, elementally charged enchantments and items.

Crafting Requirements:

* Levels: Artificer 15, Wizard 15
* Components: A small, iridescent gemstone, a piece of Xen'drik ore, and a fragment of an ancient creation pattern.
* Time: 1 week per 500 GP value of the Spark.
* Cost: 75,000 GP + 25,000 GP per 500 GP value of the Spark.
* Crafting DC: 25 + the value of the Spark in GP.

This item is only available through House Cannith, and it is a symbol of their mastery over the arcane arts. The creation of the Kedran's Spark of Xen'drik is a closely guarded secret, known only to the highest-ranking members of the house. The Spark is often used as a bargaining chip or a symbol of loyalty and allegiance amongst the members of House Cannith.

In [140]:
response = ""
for chunk in supervisor.respond("Create a Brelish war veteran"):
    response += chunk
    clear_output()
    display(Markdown(response.replace('\\n', '\n')))

Backstory: Julio Azevepo served with distinction in the Brelish army during the Last War, fighting on the western front. After the war ended in 996 YK, he found himself among the many veterans who settled on the Western Frontier. He became a member of the Grizzlies, a militia in the town of Quickstone, where he now serves as a deputy under Sheriff Constable. His military service has left him with a deep-seated suspicion of Droaamites and their allies, a belief that has occasionally led to clashes with the Mourners, kobolds, and other non-Brelish inhabitants of the area.

Long-term Goals: Julio's long-term goal is to maintain order in Quickstone and protect the town and its citizens from threats, both from within and without. He has a particularly strong desire to see Droaamites and their allies driven out of the region, and views his role in the Grizzlies as a means to that end. He also harbors a personal ambition to prove himself as a leader, both within the Grizzlies and in Breland as a whole.

Immediate Wants and Needs: Julio is currently focused on investigating a series of recent crimes in Quickstone, which he believes are being perpetrated by Droaamites or their sympathizers. He is also in need of supplies for the Grizzlies, including weapons and equipment, as well as food and other necessary supplies for the camp. He is also keen to maintain the loyalty and support of his fellow Grizzlies, and to keep them from causing trouble with the Mourners and other non-Brelish inhabitants of the area.

Personality Traits: Julio is a pragmatic and disciplined individual, with a strong sense of duty and a tendency to view problems as obstacles to be overcome rather than insurmountable challenges. He is also suspicious of outsiders, particularly those who are not Brelish, and is quick to assume the worst about them. He is fiercely loyal to Breland and Count ir'Blis, and is willing to go to great lengths to protect them and their interests.

Secrets: Julio's military service during the Last War left him with a number of scars, both physical and emotional. He has a particular a**Character Name:** Julio Azevepo

**Race:** Human (Variant Human)

**Class:** Ranger (Gloom Stalker) - Level 3

**Attributes:**
- Strength: 14
- Dexterity: 16 (with Dexterity bonus from race)
- Constitution: 14
- Intelligence: 10
- Wisdom: 16 (with Wisdom bonus from race)
- Charisma: 8

**Proficiencies:**
- Saving Throws: Wisdom, Dexterity
- Skills: Athletics, Survival, Stealth, Perception
- Tools: None
- Weapons: Simple Weapons, Martial Weapons, Shortswords, Longswords, Hand Crossbows, Longbows, Rapier
- Armor: Light Armor, Medium Armor, Shields

**Proficiency Bonus:** +3

**Hit Points:** 39 (3d10 for Ranger + 3d8 for Variant Human)

**Weapons and Equipment:**
- Longbow (with a quiver of 20 arrows)
- Short Sword
- Hand Crossbow (with 10 bolts)
- Rapier
- Leather Armor
- Explorer's Pack
- Dungeoneer's Pack
- A set of traveler's clothes
- A pouch containing 15 gold pieces
- A small, well-crafted silver locket (a personal item)

**Gloom Stalker Ranger Features:**
- Favored Enemy: Drow or Drow-like
- Shadow Tread: Advantage on Dexterity (Stealth) checks made to hide in dim light or darkness
- Stalker's Fast Movement: Can move up to half their speed when they are hidden from normal sight and have advantage on Dexterity (Stealth) checks
- Umbra Vision: Can see up to 120 feet in dim light and darkness as if it were bright light

**Spells:**
- Cantrips: Green-Flame Blade, Hunter's Mark
- 1st Level: Cure Wounds, Ensnaring Strike

**Armor Class:** 13 (Leather Armor + Dexterity modifier)

**Inventory:**
- A set of common clothes
- A bedroll
- A backpack
- A mess kit
- A waterskin
- 50 gold pieces
- A map of the Western Frontier (personal item)
- A set of thieves' tools (for investigating crimes)
- A vial of holy water (for protection against undead)
- A set of traveler's clothes
- A small, well-crafted silver locket (a personal item)

In [46]:
response = ""
for chunk in supervisor.respond("Tell me about the languages of Eberron."):
    response += chunk
    clear_output()
    display(Markdown(response.replace('\\n', '\n')))

In the world of Eberron, languages reflect culture and geography. Here are some notable languages and their typical speakers:

1. Common: The language of the Five Nations and the language of trade in Khorvaire, known by most of its people.

2. Abyssal: The common tongue of all fiends, sometimes called “Khyber’s Speech.”

3. Celestial: The language of archons of Shavarath', known as “the tongue of Siberys.”

4. Draconic: Used by dragons, kobolds, troglodytes, lizardfolk, and others.

5. Elven: Spoken by elves and drow.

6. Dwarven: Used by dwarves.

7. Giant: Used by ogres, giants, and drow.

8. Goblin: Primarily spoken by goblins, hobgoblins, and bugbears.

9. Halfling: Spoken by halflings.

10. Ignan: Used by fire-based creatures.

11. Infernal: Used by devils of Shavarath'.

12. Irial: Used by Ravids, positive energy users.

13. Kythric: Used by Slaadi, chaotic outsiders.

14. Mabran: Used by Nightshades, shadows, and creatures of Mabar.

15. Orc: Spoken by orcs.

16. Quori: Used by the Inspired, kalashtar, and other Quori.

17. Riedran: Used by the lower classes of Sarlona.

18. Sylvan: Spoken by dryads, eladrins, and creatures of Thelanis.

19. Terran: Used by Xorns and other earth-based creatures.

20. Undercommon: Used by chokers, underground Daelkyr denizens.

Eberron is the language of Lamannia and the natural world, and the Druidic language is derived from Eberral as well. The Orc language is written using the Goblin script, and the Goblin language was the trade language of the goblin empire of Dhakaan. Common is the language of trade in Khorvaire.

In [245]:
response = ""
for chunk in supervisor.respond("Create a Brelish war veteran"):
    response += chunk
    clear_output()
    display(Markdown(response.replace('\\n', '\n')))

Backstory:
Cameron Calamity Jones served as a Wandslinger in the Brelish army during the Last War. He was a prodigy with arcane magic, selected for the elite Wandslinger unit at a young age. He quickly rose through the ranks, gaining the reputation as the "Calamity" for his fearless battle style and his uncanny ability to turn the tide of a battle with a well-placed spell.

As the war raged on, Cameron became disillusioned with the political machinations that drove the conflict and began to question the motives of his superiors. He grew increasingly frustrated with the lack of strategic thought and the focus on attrition rather than victory.

In the final years of the war, Cameron and his fellow Wandslingers were sent on a covert mission deep into enemy territory. Their objective was to sabotage an enemy castle and gather intelligence. The mission was a success, but the cost was high; Cameron lost several comrades, including his best friend.

The aftermath of the war found Cameron struggling to reconcile his idealism and his experiences on the battlefield. He returned to Quickstone, where he found the town had changed dramatically. Count ir'Blis, who had been a strong supporter of the war, seemed less concerned with rebuilding and more focused on consolidating power.

Cameron joined the Grizzlies, hoping to make a difference in the community and to help keep the peace. However, he remains haunted by the memories of the war and the lives lost. He has not yet found a way to move on, and he continues to struggle with feelings of guilt and anger.

Long-term Goals:
Cameron's long-term goal is to find a way to put his magical abilities to use for the greater good. He believes that his skills could be valuable in maintaining peace and justice in Quickstone and the wider Breland. However, he is still unsure how best to accomplish this and spends a lot of time considering his options.

Immediate Wants and Needs:
Cameron's immediate wants include finding a purpose in Quickstone and feeling a sense of belonging. He also needs to find a way to move on from his experiences in the war and to come to terms with his feelings of guilt and anger. His immediate needs include housing, food, and equipment, asCharacter Name: Cameron Calamity Jones

Race: Human (Variant: Brelish)

Class: Sorcerer (Draconic Bloodline: Bronze) - Level: 3

Attributes:
- Strength: 12 (+1)
- Dexterity: 16 (+3) (Base: 13, Armor: 3)
- Constitution: 14 (+2)
- Intelligence: 16 (+3)
- Wisdom: 12 (+1)
- Charisma: 18 (+4)

Proficiencies:
- Saving Throws: Intelligence, Wisdom
- Skills: Arcana, Deception, History, Insight, Persuasion

Proficiency Bonus: +3

Feats and Proficiencies:
- War Caster (Sorcerer)
- Light Armor Proficiency (Human)

Hit Points: 45 (3d8 + 18 for Sorcerer, 1d10 + 10 for Human)

Weapons and Equipment:
- Quarterstaff (Proficient, Attack Bonus: +5)
- Dagger (Proficient, Attack Bonus: +4)
- Spellcasting Focus (Amber Amulet)
- Explorer's Pack
- Leather Armor (AC: 13)
- 10 Potions of Healing (1d4+2 HP each)
- 20 Gold Pieces
- A worn, tattered map of Breland (Personal Item)
- A small, bronze dragon figurine (Lucky Charm)

Class and Level: Sorcerer 3

Attack Roll: +5 (Quarterstaff) / +4 (Dagger)

Armor Class: 16 (AC: 13, Dexterity: 3)

Spells:
- Cantrips: Mage Hand, Firebolt, Prestidigitation
- Level 1: Burning Hands, Chromatic Orb, Shield, Thunderwave
- Level 2: Darkness, Scorching Ray, Invisibility

Spell Slots:
- Level 1: 4 slots (2 at 1st level, 1 at 2nd level, 1 at 3rd level)
- Level 2: 2 slots (1 at 2nd level, 1 at 3rd level)

Long-term Goals: To find a way to put his magical abilities to use for the greater good in Quickstone and the wider Breland.

Immediate Wants and Needs: To find a purpose in Quickstone, feel a sense of belonging, and come to terms with his feelings of guilt and anger from the war.

In [190]:
response = ""
async for chunk in supervisor.respond("Create a Brelish war veteran"):
    response += chunk
    clear_output()
    display(Markdown(response.replace('\\n', '\n') + '\n\n\n'))

**Character Name:** Cameron Calamity Jones

**Backstory:** Cameron Jones was born and raised in the bustling city of Sharn, Breland. Growing up, he was always drawn to the arcane and the magical, fascinated by the wandslingers, the new breed of soldiers who could harness magic on the battlefield. When the Last War broke out, he saw an opportunity to serve his nation and follow his passion. He enlisted in the Brelish army and was trained as a wandslinger, quickly rising through the ranks thanks to his natural aptitude for arcane combat.

During the war, Cameron distinguished himself with acts of bravery and tactical genius. His unit, the Lion's Roar, became known for their daring charges and fearsome power on the battlefield. However, the war took its toll on Cameron, both physically and emotionally. He lost many close comrades, and the horrors of war haunt his dreams to this day.

At the end of the war, Cameron was honorably discharged and returned to Sharn. He initially struggled to adapt to civilian life, feeling lost without the structure and purpose that the army provided. But his love for magic and his desire to help his nation never waned. He used his skills and connections to become a renowned wandslinger for hire, often taking on dangerous jobs that others would shun.

**Long-term Goals:** Cameron's long-term goal is to prove that wandslingers are more than just battlefield warriors. He hopes to use his skills to help rebuild Breland and make it a safer, more prosperous place. He also aspires to establish a school or academy for young wandslingers, teaching them the arts of arcane combat and instilling in them a sense of duty and honor.

**Immediate Wants and Needs:** In the short term, Cameron is seeking to establish himself as a reliable and highly skilled wandslinger for hire. He needs to find steady work that allows him to pay his bills and maintain his equipment. He is also looking to reconnect with old comrades from the Lion's Roar, to share stories and reminisce about the good old days.

**Personality Traits:** Cameron is fiercely loyal to Breland and its people. He is brave, resolute, and unyielding in the**Character Name:** Cameron Calamity Jones

**Race:** Human (Variant: Tiefling)

**Class:** Wandslinger (Homebrew Class) - Level 5

**Attributes:**
- Strength: 14
- Dexterity: 16 (Dexterity Modifier: +3)
- Constitution: 14
- Intelligence: 16
- Wisdom: 12
- Charisma: 14

**Proficiencies:**
- Armor Proficiency: Light armor, medium armor, shields
- Weapon Proficiency: Simple weapons, martial weapons, firearms
- Tool Proficiency: Artisan's tools (blacksmith's tools), thieves' tools
- Skill Proficiency: Athletics, Acrobatics, Arcana, History, Investigation, Perception, Stealth

**Proficiency Bonus:** +3

**Saving Throws:** Intelligence, Dexterity

**Feats and Proficiencies:**
- Shield Master (Feat)
- Wandslinger's Focus (Class Feature)

**Hit Points:** 52 (10 HP from level 5 Wandslinger, 42 HP from 14 CON)

**Weapons and Equipment:**
- Rapier (Longsword for melee combat when using a shield)
- Shield
- Wand (Quarterstaff for ranged combat)
- 20 Bullets (for firearm)
- Explosive Runes (5)
- Arcane Focus (Quarterstaff)
- Leather Armor
- Adventurer's Pack
- Explorer's Pack
- 50 Gold Pieces
- A small, intricately crafted silver locket containing a picture of the Lion's Roar unit
- A set of custom-made blacksmith's tools, a gift from a comrade

**Attack Bonus:**
- Rapier: +6 to hit, 9 (1d8 + 4) slashing damage
- Longsword: +6 to hit, 11 (1d10 + 4) slashing damage
- Shield: +2 AC bonus
- Wand (Quarterstaff): +6 to hit, 15 (4d6) force damage (or 1d10 + 4 force damage if using Explosive Runes)

**Class and Level:** Wandslinger 5

**Armor Class:** 16 (13 + 3 Dexterity Modifier)

**Spellcasting Ability:** Arcane Tradition (Homebrew Subclass) - Intelligence

**Spells:**
- Cantrips: Arcane Charge, Mage Hand, Mending, Message, Prestidigitation, Thaumaturgy
- 1st Level: Burning Hands, Cure Wounds, Detect Magic, Magic Missile, Shield
- 2nd Level: Darkvision, Enhance Ability, Hold Person, Scorching Ray, Web

**Inventory:**
- A set of fine clothes for formal occasions
- A map of Breland, marked with the locations of old comrades from the Lion's Roar
- A journal to document his adventures and experiences
- A set of tools for repairing and modifying his wands and equipment
- A small, enchanted amulet that grants

In [171]:
response = ""
async for chunk in supervisor.respond("Create a Brelish war veteran"):
    response += chunk
    clear_output()
    display(Markdown(response.replace('\\n', '\n')))

Backstory:
Cameron Calamity Jones is a Brelish war veteran who served as a wandslinger during the Last War. Originally a non-combatant with a background in arcane studies, Cameron was drafted into the Brelish military and trained to wield a wand in combat. He quickly rose through the ranks, earning a reputation as a formidable and relentless combatant, earning him the nickname "Calamity" from his peers. After the war, Cameron returned to Breland and settled in Quickstone, taking up a position as a sheriff's deputy and using his skills as a wandslinger to maintain law and order in the town and surrounding areas.

Long-term Goals:
Cameron's long-term goal is to one day be appointed as the sheriff of Quickstone and build a strong and respected force to protect the town and its citizens. He also hopes to build a peaceful and prosperous future for Breland, and to help the nation rebuild after the destruction caused by the Last War.

Immediate Wants and Needs:
Cameron is currently focused on maintaining law and order in Quickstone, and is always on the lookout for troublemakers and criminals who threaten the peace. He is also in constant need of supplies and equipment to maintain his wand and keep it functioning properly, as well as medical supplies to treat any injuries sustained during his duties.

Personality Traits:
Cameron is a disciplined and driven individual, with a strong sense of duty to his country and his community. He is also fiercely independent and stubborn, and is not afraid to stand up for what he believes in, even if it means going against authority. He is deeply protective of those he cares about, and will stop at nothing to protect them from harm.

Secrets:
Cameron has a dark secret from his time in the war: he was responsible for the death of a fellow soldier who had threatened to reveal a scandal involving a high-ranking officer. Cameron has spent years living with the guilt of this act, and is haunted by the memory of his former comrade. He has also been secretly training a young Brelish soldier, believing that he can help the young man reach his full potential and become a valuable asset to the Brelish military.

Relevant D&D 5e Skill Checks:
Players may be able to notice Cameron's proficiency with Smith's Tools, as he is constantly maintaining and repairing his wand. They may also be able to spot that he is a wandslinger by observing his wand at rest on his belt, or by witnessing him casting spells during combat. With a successful Investigation check, players may be able to discover Cameron's dark secret, or overhear him talking to his secret trainee. With a successful Persuasion check, players may be able to convince Cameron to look the other way in certain situations, or to provide them with information or assistance.Character Name: Cameron Calamity Jones

Race: Human (Variant Human)

Class: Wandslinger (Homebrew Class) - Level 3

Attributes:
- Strength: 12 (+1)
- Dexterity: 16 (+3)
- Constitution: 14 (+2)
- Intelligence: 16 (+3)
- Wisdom: 12 (+1)
- Charisma: 14 (+2)

Proficiencies:
- Armor Proficiency: Light Armor
- Weapon Proficiency: Simple Weapons, Martial Weapons, Wands
- Tool Proficiency: Smith's Tools
- Skill Proficiency: Athletics, Investigation, Persuasion, Perception

Proficiency Bonus: +3

Saving Throws: Wisdom, Charisma

Feats and Proficiencies:
- Feat: Sharpshooter (Dexterity)
- Proficiency: Resilient (Constitution)

Hit Points: 39 (3d8 for Human + 3d10 for Wandslinger)

Weapons and Equipment:
- Wand of Fireball (3 charges)
- Wand of Magic Missile (4 charges)
- Wand of Cure Wounds (2 charges)
- Short Sword (proficient)
- Leather Armor (armor class 11 + Dexterity modifier)
- Explorer's Pack
- Smith's Tools
- A set of traveler's clothes
- A set of Brelish military uniform (for formal occasions)
- A small pouch containing 15 gold pieces
- A locket with a picture of his family (personal item)

Attack Bonus:
- Short Sword: +5 (Dexterity modifier + Proficiency Bonus)

Class and Level Features:
- Wandslinger: Wand Mastery, Wand Channeling, Wand Specialization (Fire)
- Level 3: 3rd-level Spell Slots (2 slots of 3rd level)

Spells:
- Cantrips: Mage Hand, Prestidigitation, Thaumaturgy
- 1st Level: Burning Hands, Chromatic Orb, Cure Wounds, Detect Magic, Shield, Thunderwave
- 2nd Level: Darkness, Enhance Ability, Hold Person, Scorching Ray, Web
- 3rd Level: Fireball, Fly, Hypnotic Pattern, Lightning Bolt, Vampiric Touch

Armor Class: 14 (Leather Armor + Dexterity modifier)

Inventory:
- A set of spare wand components (for crafting new wands)
- A spellbook containing spells from the Wandslinger class
- A set of lockpicks (personal item)
- A map of Breland (personal item)
- A letter from a former comrade (personal item)
- A small pouch containing 50 gold pieces (for supplies and equipment)
- A set of manacles (for apprehending criminals

# Not Working - This is for architecture for an Autonomous Co-DM

In [141]:
class SupervisorAgent(Agent):
    version = '01'
    base_llm = None

    def listen(self, context: str) -> bool:
        """This agent does not listen to any contextual information"""
        return False
    
    def _retrieve(self, q: str):
        """This agent does not retrieve any contextual information."""
        return []

    def _prompt(self, q: str):
        """This agent does not require a prompt."""
        return ""

    def respond(self, q: str) -> Iterable[str]:
        
        category = self.receive(self.ask(request_classifier, q))

        q = re.sub(r'^[a-zA-Z0-9]+ +', '', q)
        
        if category == 'lookup':
            yield from self.ask(canonical_summary_agent, q)
        elif category == 'original':
            type_of_content = self.receive(self.ask(request_classifier, q))
            if type_of_content == 'encounter':
                yield from self.ask(encounter_creator_agent, q)
            elif type_of_content == 'settlement':
                yield from self.ask(encounter_creator_agent, q)
            elif category == 'character':
                yield from self.ask(character_prompter_agent, q)
                yield from self.ask(character_generator_agent, self.received_response)
          
    def run(self):
        # Start when you start a session. One Agent needs to run per campaign.
        self.campaign_history = self.receive(self.ask(history_agent, """Give a brief overview of the campaign so far."""))
        self.party_makeup = self.receive(self.ask(party_agent, """Give an overview of the party so far. 
                                                                  Who are the characters?
                                                                  Give a two-three sentence about the party, including their class and level."""))
        previously_suggested_prompts = []
        while True:
            current_session_context = self.receive(self.ask(session_agent, q))
            if session_ended:
                self.tell(history_agent, f"""Here is what happened during the session: {current_session_context}""")
                break

            current_location_context = self.receive(self.ask(session_agent, q))
            dm_suggestion = self.receive(self.ask(co_dm_agent, f"""
            Based on the history & the current location, what might the DM need at the moment?
            Create one prompt. Make sure it is different than the previous suggestions.

            Current campaign so far:
            {self.campaign_history}

            Party makeup:
            {self.party_makeup}

            Current location:
            {current_location_context}

            Already suggested prompts, do not repeat these:
            {' /// '.join(previously_suggested_prompts)}

            What might the DM need? Suggest a prompt:
            """))


            previous_suggetion_summary = self.receive(self.ask(summary_agent, dm_suggestion)
            previously_suggested_prompts.append(dm_suggestions)
            previously_suggested_prompts = previously_suggested_prompts[:3]

            yield self.respond(dm_suggestion)
            
            sleep(30)



In [ ]:
q = "Tell me about the languages of Eberron."

In [206]:
%%time
for chunk in supervisor.respond("Tell me about the languages of Eberron."):
    display(chunk)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'lookup'

CPU times: user 982 ms, sys: 169 ms, total: 1.15 s
Wall time: 1.15 s


In [207]:
%%time
for chunk in supervisor.respond("Create a House Cannith item."):
    display(chunk)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'character'

CPU times: user 742 ms, sys: 146 ms, total: 888 ms
Wall time: 886 ms


In [ ]:
# TODO: class ShortResponseAgent(Agent)
# TODO: class CharacterCreatorAgent(Agent)
# TODO: class CharacterDetailerAgent(Agent)
# TODO: class OriginalContentAgent(Agent)

In [35]:
import networkx

In [94]:
q = "Tell me about fashion in the five nations."
response = get_info(q)
print(response)

[2025-01-20T04:24:22Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
[2025-01-20T04:24:22Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
[2025-01-20T04:24:22Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
[2025-01-20T04:24:22Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


1. During the Last War, the fashion styles of the Five Nations evolved to distinguish their soldiers and improve their tools of war. Common soldiers typically wore light, medium, or heavy armor made of leather, metal, or a combination of both.

    2. Elite forces, officers, and mercenaries used different styles and materials. Each nation had its own distinct approach to fashion, both in its armor and civilian clothing.

    3. Beyond practical considerations, adventurers might choose to wear specific clothing as a cultural expression. For example, a Cyran might wear gloves and masks, a Karrn might wear gazyrs sewn onto leather armor, an Aundairian might wear a cloak with strange proportions, a Thrane might wear clothes covered in embroidered patches, an Aundairian might wear clothes covered in embroidered patches, and a Brel might wear pointy-toed shoes and long puffy sleeves.

    4. Many contemporary fashion designers now produce lines of fashion specifically for adventurers.

    5

In [97]:
q = "Tell me about fashion in the five nations."
response = get_info(q)
print(response)

[2025-01-20T04:25:16Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
[2025-01-20T04:25:16Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
[2025-01-20T04:25:16Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
[2025-01-20T04:25:16Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


1. The common armor of today for the Five Nations include light, medium, and heavy armor. These typically involve a metal helmet, breastplate for medium armor, and leather or chainmail for light armor. However, each nation has its distinct approach to fashion both in its armor and civilian clothing.

    2. Adventurers from the Five Nations often represent their homeland in their clothing, although not always for practicality. Adventurers are an important part of each nation, with adventuring guilds and independents playing crucial roles in their cultures.

    3. The Church of the Silver Flame, one of the most significant religions in the Five Nations, is recognized by its delicately embroidered, silver-dyed denim robes. Their vestments are sewn with extra space for mobility, and are usually closed with toggles rather than fasteners.

    4. In Middle Tavick's Landing, one can find fusion of Brelish lapels, Thrane cloaks, and other styles, as it serves as the melting pot of cultures i

In [99]:
q = "Suggest an outfit for a dragonmarked heir."
response = get_info(q)
print(response)

[2025-01-20T04:26:31Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
[2025-01-20T04:26:31Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
[2025-01-20T04:26:31Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
[2025-01-20T04:26:31Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


1. Light Armor: Because the dragonmark heir does not have proficiency in heavy armor, a light armor will offer better protection while being able to move easily. Many Mror wear decorative armor that uses the statistics of light armor, but evokes the general flavor of a heavier breastplate. This will allow the heir to retain the martial aspect to their attire that the Mror generally favor, but still be able to move freely.

    2. Robe or Tunic: To balance out the armor, a long robe or tunic can be used as the top garment. This will not only provide a more elegant look, but also offer more coverage for the dragonmark on their arm/chest.

    3. Sash or Belt: A sash or belt can be worn around the waist to hold up the robe/tunic and also add a bit of color and personality to the outfit.

    4. Boots: Boots would be a practical choice for footwear, as they offer protection and support for the feet while moving.

    5. Accessories: To complete the outfit, the heir could wear various acces

In [100]:
!nvidia-smi --query-gpu=memory.total,memory.used,memory.free --format=csv

memory.total [MiB], memory.used [MiB], memory.free [MiB]
15360 MiB, 10967 MiB, 4129 MiB


In [36]:
def get_one_word_answer(q: str) -> str:
    results = retrieve(q, embedding_model, 5, 'cosine')

    retrieved_text = dedent("""
    -----
    """).join(results['text'])

    prompt = f"""
    [INST]
    Use the following information (until the final cutoff =====) to answer the user query Q below.
    The answer has to be one word only.
    You are trying to get the most accurate answer in one shot.

    {retrieved_text}

    =====

    Q:
    {q}

    A:
    [/INST]
    """
    print(len(prompt))

    model_inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

    generated_ids = model.generate(**model_inputs, 
                                   max_length=50,
                                   max_new_tokens=256, 
                                   do_sample=True, 
                                   eos_token_id=tokenizer.eos_token_id)

    response = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    return (response.split('A:\n')[1] if 'A:\n' in response else response).strip()


In [37]:
q = "What is the name of the Breland veterans community in Quickstone?"
response = get_one_word_answer(q)
print(response)

[2025-01-21T01:40:31Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
[2025-01-21T01:40:31Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
[2025-01-21T01:40:31Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
[2025-01-21T01:40:31Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` w

8460
Tents


In [ ]:
q_and_a = [
    ("What is the name of the prominent university in Sharn?", "Morgrave"),
    ("What type of magic does Karrnath use in its military?", "Necromancy"),
    ("Who leads Kind's Dark Lanterns?", "Vron"),
    ("Who leads King's Dark Lanterns?", "Vron"),
    ("What is the name of the Breland veterans community in Quickstone?", "Tents"),
]

In [38]:
# Required globals: model, embedding_model, tokenizer, retreive. I might also need a score or k or other parameters.
retrieve = retrieve_baseline
def generate_character(q: str) -> str:
    results = retrieve(q, embedding_model, 10, 'cosine')

    retrieved_text = dedent("""
    -----
    """).join(results['text'])

    prompt = f"""
    [INST]
    Use the following information (until the final cutoff =====) to create a character based on the 5th Edition rules for the world of Eberron.
    Use the prompt P as a starting point.
    Give the complete character sheet.
    A complete character sheet should include name, race, class, level, attributes, proficiencies based on the class and fifth edition rules, 
    proficiency bonus based on class and level, saving throws bonus based on class, feats and proficiency , hit points based on class and level, 
    weapons and equipment, attack bonus for each weapon based on abilities, any race or class based features, class and level, attack roll, 
    and equipment, armor class based on dexterity and armor, and if the character has spellcasting ability, spells based on class and level, 
    as well as the number of spell slots. 
    Include an inventory based on their level, class, and race, with a few personal touches in the items.
    
    Also include a compelling and detailed backstory of two paragraphs related to the world of Eberron. 
    This background should include their nation.
    Include another short paragraph about their intentions.
    Include one other short paragraph about their tactics in combat.
    Finally, include one paragraph for the visuals.

    {retrieved_text}

    =====

    Prompt P:
    {q}

    Character:
    [/INST]
    """

    model_inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

    generated_ids = model.generate(**model_inputs, 
                                   max_new_tokens=2048, 
                                   do_sample=True, 
                                   eos_token_id=tokenizer.eos_token_id)

    response = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    return (response.split('Character:\n')[1] if 'Character:\n' in response else response).strip()

    

In [103]:
character = generate_character("War veteran")
print(character)

[2025-01-20T04:34:02Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
[2025-01-20T04:34:02Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
[2025-01-20T04:34:02Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
[2025-01-20T04:34:02Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


8. Character: Caelum "Cal" Iarwain
    9. Race: Human
    10. Class: Ranger 1
    11. Attributes (5e standard array): Str 12, Dex 14, Con 10, Int 8, Wis 12, Cha 10
    11. Class Abilities:
        1. Starting Proficiencies: Perception, Survival
        2. Level 1 Fighting Style: Archery
        3. Ranger Spells: Cat's Grace, Hunter's Mark
        4. Favored Enemy: Orcs
        5. Natural Explorer: Swamp, Urban
        6. Skills: Climb, Dexterity, Intelligence, Investigation, Nature, Navigation, Survival, Persuasion
        7. Tools: None
    12. Proficiency Bonus (5e default): +2
    13. Saving Throws:
        1. Wisdom (Race)
        2. Dexterity (Class)
    14. Feats & Levels: None
    15. Hit Points: 10 (Levels: 1 Ranger) + Hit Die (next level: 1d10)
    16. Weapons & Equipment:
        1. Shortbow
        2. Quiver (20 arrows)
        3. Short Sword
        4. Leather Armor
    17. Attack Bonus (Per Stat):
        1. Shortbow: +5 to hit (1d8 damage)
        2. Short Sword: +5 to hi

In [40]:
character = generate_character("Comin relief, similar to C3p0")
print(character)

[2025-01-21T01:43:24Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
[2025-01-21T01:43:24Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
[2025-01-21T01:43:24Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
[2025-01-21T01:43:24Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Name: RedLight Disiuct p
     Race: Golem (Grey Iron)
     Class: Sorcerer (Harbinger of Faith)
     Level: 5
     Attributes:
       Strength: 14 (+2)
       Dexterity: 12 (+1)
       Constitution: 14 (+2)
       Intelligence: 16 (+3)
       Wisdom: 8 (-1)
       Charisma: 18 (+4)
     Proficiencies: Arcana, History, Insight, Religion
     Proficiency Bonus: +3
     Saving Throws: Wisdom (+0), Charisma (+6)
     Feats and proficiencies:
       - Mage Armor (1 level sorcerer)
       - Spellcasting (5 levels sorcerer)
     Hit Points: 37 (5d8 for Golem + 21 (5d6 + 5 from sorcerer levels))
     Weapons and equipment:
       - Dagger (attack roll +5)
       - Quarterstaff (attack roll +4)
       - Tindertwig (no attack bonus, helps with igniting objects)
       - A necklace with a charm of healing (1d4 + 2 HPs)
       - A pouch containing 2d4 sprigs of Pacifycus, used to reduce fevers
       - A small wooden effigy of Sovereign Host, used as an additional focus in spells
     Armor Class:

In [39]:
character = generate_character("War veteran 3rd level")
print(character)

[2025-01-21T01:41:52Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
[2025-01-21T01:41:52Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
[2025-01-21T01:41:52Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
[2025-01-21T01:41:52Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Name: Cameron Calamity Jones
     Race: Human
     Class: Wandslinger (Blaster)
     Level: 3

    Attributes:
     - Strength (Str): 10 (+0)
     - Dexterity (Dex): 14 (+2)
     - Constitution (Con): -1 (-2)
     - Intelligence (Int): 14 (+2)
     - Wisdom (Wis): 12 (+1)
     - Charisma ( Cha): 10 (+0)

    Proficiencies:
     - Armor: Light armor, medium armor, shields
     - Weapons: Simple weapons, martial weapons
     - Skills: Arcana +4, Athletics +2, Medicine +3

    Proficiency Bonus: +2
     Saving Throws Bonus: Dex +4, Con +4

    Hit Points: 39 (6d10 + 12)

    Equipment:
     - Chain Shirt
     - Heavy Crossbow with 20 bolts
     - Two Longswords
     - Studded Leather Armor
     - Dungeoneer's pack
     - Leather armor, shield, and 14 bolts (equipped)
     - A wand (your magical focus) that can cast spells from the Enchantment and Evocation spell lists

    Attack Bonus:
     - Longsword: +4
     - Heavy Crossbow: +3

    Class Features:
     - Blaster: Your choice grants 

In [108]:
character = generate_character("5th level gunslinger")
print(character)

[2025-01-20T04:42:21Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
[2025-01-20T04:42:21Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
[2025-01-20T04:42:21Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
[2025-01-20T04:42:21Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


5th-level Wandslinger (Blaster)
    Medium humanoid (human)
    Str 10, Dex 14, Con 14, Int 16, Wis 12, Cha 10,
    Saving throws Dex +5, Con +5
    Skills Arcana +6, Athletics +3, Medicine +4
    Hit Dice: 5d10
    Proficiencies: Light armor, Medium armor, Shields; Simple weapons, Martial weapons
    Senses: passive Perception 11
    Languages: Common, Dwarvish

    Attributes:
    Attack bonus (longbow): +7
    Armor Class: 17

    Special Features:
    True Shot (Gunslinger): You can take a −4 penalty to the attack roll to gain a +10 bonus to the damage roll for that attack.
    Assess Foe (Gunslinger): By taking a minute of time to examine your adversary, you can learn information about the adversary and make special moves to gain an advantage in battle.
    Magic Ammunition (Gunslinger): You learned to cast spells as a bonus action without using a spell slot and can now imbue your quiver of arrows with a magical power. When you activate this power, select one spell from the list b

# Test Langchain

In [21]:
!pip install langchain-huggingface --upgrade

In [25]:
from langchain_huggingface.llms import HuggingFacePipeline
from langchain_core import messages

In [23]:
!nvidia-smi --query-gpu=memory.total,memory.used,memory.free --format=csv

memory.total [MiB], memory.used [MiB], memory.free [MiB]
15360 MiB, 4087 MiB, 11009 MiB


['AIMessage',
 'AIMessageChunk',
 'AnyMessage',
 'BaseMessage',
 'BaseMessageChunk',
 'ChatMessage',
 'ChatMessageChunk',
 'FunctionMessage',
 'FunctionMessageChunk',
 'HumanMessage',
 'HumanMessageChunk',
 'InvalidToolCall',
 'MessageLikeRepresentation',
 'RemoveMessage',
 'SystemMessage',
 'SystemMessageChunk',
 'ToolCall',
 'ToolCallChunk',
 'ToolMessage',
 'ToolMessageChunk',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_message_from_dict',
 'ai',
 'base',
 'chat',
 'convert_to_messages',
 'convert_to_openai_messages',
 'filter_messages',
 'function',
 'get_buffer_string',
 'human',
 'merge_content',
 'merge_message_runs',
 'message_chunk_to_message',
 'message_to_dict',
 'messages_from_dict',
 'messages_to_dict',
 'modifier',
 'system',
 'tool',
 'trim_messages',
 'utils']

In [ ]:

character_prompter = HuggingFacePipeline.from_model_id(
    model_id=model_path,
    task="text-generation",
    pipeline_kwargs=dict(
        max_new_tokens=512,
        do_sample=True,
        eos_token_id=tokenizer.eos_token_id,
    ),
)



In [ ]:

character_generator = HuggingFacePipeline.from_model_id(
    model_id=model_path,
    task="text-generation",
    pipeline_kwargs=dict(
        max_new_tokens=512,
        do_sample=True,
        eos_token_id=tokenizer.eos_token_id,
    ),
)



In [ ]:
# from langchain_huggingface.llms import HuggingFacePipeline
# from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# model_id = "gpt2"
# tokenizer = AutoTokenizer.from_pretrained(model_id)
# model = AutoModelForCausalLM.from_pretrained(model_id)
# pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=10)
# hf = HuggingFacePipeline(pipeline=pipe)

In [ ]:
# from langchain_core.prompts import PromptTemplate

# template = """Question: {question}

# Answer: Let's think step by step."""
# prompt = PromptTemplate.from_template(template)

# chain = prompt | hf

# question = "What is electroencephalography?"

# print(chain.invoke({"question": question}))